In [23]:
import pandas as pd
from utils.output_analysis import plot_player_value_trends, save_output_tables

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from typing import List
import plotly.express as px
from config import features, test_start, target
from utils.model import test_train_split, define_features, analysis_result, prepare_future_year_data

import uuid
import datetime 
import time

In [24]:
start = time.time()
version  = "v0.0.1"

In [25]:
pdf_mvp = pd.read_csv("data/intermediate/time_series_model_data_prep.csv")

In [26]:



def predict_future_values(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    prediction_years: List[int],
    features: List[str],
    max_depth: int = None  # Not used
) -> tuple[List[pd.DataFrame], LinearRegression, pd.Series]:
    """
    Predicts future player values using Linear Regression with standardized features.

    Returns:
        - List of yearly predicted DataFrames
        - Trained LinearRegression model
        - Actual values from base test year (for evaluation)
    """

    # Scale features
    scaler = StandardScaler()
    X_train_raw = train_df[features]
    X_train_scaled = scaler.fit_transform(X_train_raw)
    y_train = train_df[target]

    # Fit regression model
    regressor = LinearRegression()
    regressor.fit(X_train_scaled, y_train)

    predictions = []
    current_df = test_df.copy()

    for i, year in enumerate(prediction_years):
        # Handle age-based features
        current_df["age"] = current_df["age_last_year"] + 1
        current_df["age_from_peak"] = (current_df["age"] - 25) ** 2

        # Decrease contract years left by 1, not below 0
        if "contract_years_left" in current_df.columns:
            current_df["contract_years_left"] = (current_df["contract_years_left"] - 1).clip(lower=0)

        X_test_raw = current_df[features]
        X_test_scaled = scaler.transform(X_test_raw)

        current_df["predicted_value"] = regressor.predict(X_test_scaled)
        current_df["year"] = year

        if i == 0:
            y_test = current_df[target]
            analysis_result(current_df, y_test, year, target)
        # Prepare next year’s data
        current_df["age_last_year"] = current_df["age"]
        current_df["value_last_year"] = current_df["predicted_value"]

        # Save results (including contract years if available)
        cols_to_keep = ["player_id", "year", "age", "predicted_value", "age_from_peak"]
        if "contract_years_left" in current_df.columns:
            cols_to_keep.append("contract_years_left")

        predictions.append(current_df[cols_to_keep].copy())

        if i < len(prediction_years) - 1:
            current_df = prepare_future_year_data(current_df)


    return predictions, regressor, y_test


In [27]:
#TODO - This is temporary fix
pdf_mvp[["team_ppg", "team_goal_difference", "team_goals_scored", "team_goals_conceded"]] = (
    pdf_mvp[["team_ppg", "team_goal_difference", "team_goals_scored", "team_goals_conceded"]].fillna(0)
)


In [28]:

features = define_features(pdf_mvp, features)

pdf_train, pdf_test = test_train_split(pdf_mvp, test_start)

In [29]:


predicted_dfs, regressor, y_test = predict_future_values(
    pdf_train, 
    pdf_test,
    prediction_years=[2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038],
    features=features
)

forecast_df = pd.concat(predicted_dfs, ignore_index=True)

2023 RMSE: 6.44
2023 R²: 0.836


In [30]:
# Print coefficients
coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": regressor.coef_
}).sort_values(by="Coefficient", key=abs, ascending=False)

print("\n🔎 Linear Regression Coefficients:")
print(coef_df.to_string(index=False))


🔎 Linear Regression Coefficients:
                  Feature   Coefficient
  subpos_Central Midfield  1.181015e+13
    subpos_Centre-Forward  9.516351e+12
subpos_Defensive Midfield  8.907077e+12
subpos_Attacking Midfield  7.174904e+12
           pos_Goalkeeper  7.037091e+12
             pos_Defender  6.747465e+12
      subpos_Right Winger  6.109785e+12
       subpos_Centre-Back  5.605286e+12
       subpos_Left Winger  5.504963e+12
        subpos_Right-Back  4.065359e+12
     subpos_Left Midfield  4.053734e+12
         subpos_Left-Back  3.589590e+12
    subpos_Right Midfield  2.852409e+12
        subpos_Goalkeeper  2.687274e+12
             pos_Midfield -2.286779e+12
    subpos_Second Striker  1.873380e+12
        team_goals_scored -1.407437e+12
      team_goals_conceded  1.389713e+12
     team_goal_difference  9.802367e+11
       goal_contributions -8.744784e+11
               pos_Attack  7.532547e+11
                    goals  5.488049e+11
           contrib_per_90  4.420332e+11
     

In [31]:
# Merge with actual market values for evaluation

merged = forecast_df.merge(pdf_test[["player_id", "name"]], on=["player_id"], how="left")
end = time.time()
diff = end - start

In [32]:
players = [418560, 357662, 565822, 859951, 148455, 274839, 542586]
plot_player_value_trends(pdf_train, merged_df=merged, player_ids=players)

In [33]:
output_df = merged.query("year == 2023")
output_df["actual_value"] = y_test.values


C:\Users\tomkn\AppData\Local\Temp\ipykernel_17596\2121820575.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
output_df["model_output_id"] = uuid.uuid4()
output_df["model_run_date"] = datetime.datetime.now()
output_df["time_taken_seconds"] = diff
output_df["features_used"] = str(features)
output_df["model_type"] = "Regression"
output_df["split_year"] = test_start
output_df["version"] = version

C:\Users\tomkn\AppData\Local\Temp\ipykernel_17596\770660196.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tomkn\AppData\Local\Temp\ipykernel_17596\770660196.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tomkn\AppData\Local\Temp\ipykernel_17596\770660196.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [35]:
save_output_tables(output_df)